# Linear Algebra Refresher for Robotics Kinematics (Interactive)

This notebook is a **Binder-ready** refresher on the linear algebra you’ll use constantly in robotics kinematics:
- vectors and vector geometry
- dot product and projection
- matrices and matrix multiplication
- linear transforms (rotation, scaling, shear)
- **homogeneous transformation matrices** in **2D** and **3D**
- rigid-body transforms and changing coordinates between reference frames

Throughout, use sliders to build intuition by seeing the math move points, vectors, and coordinate frames.

---


## Environment sanity check

Run this cell first. It checks that required packages are installed and that widgets/Plotly render.


In [ ]:
import importlib

required = ["numpy", "matplotlib", "ipywidgets", "plotly"]
optional = ["sympy", "anywidget"]

print("Checking required packages...")
missing = [p for p in required if importlib.util.find_spec(p) is None]
for p in required:
    print(("✓ " if p not in missing else "✗ ") + p)

if missing:
    raise ImportError(f"Missing required packages: {missing}")

print("\nChecking optional packages...")
for p in optional:
    print(("✓ " if importlib.util.find_spec(p) is not None else "○ ") + p)

print("\nWidget test (you should see a slider):")
import ipywidgets as widgets
from IPython.display import display
display(widgets.IntSlider(description="Widget test"))

print("Plotly test (you should see an interactive 3D line):")
import plotly.graph_objects as go
fig = go.FigureWidget(data=[go.Scatter3d(x=[0,1], y=[0,1], z=[0,1], mode="lines+markers")])
fig.update_layout(title="Plotly sanity check", margin=dict(l=0,r=0,t=40,b=0))
display(fig)

print("\nSanity check complete.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown
import plotly.graph_objects as go


## Plot helpers (2D)

In [ ]:
def setup_2d(ax, lim=3):
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.grid(True, alpha=0.3)
    ax.axhline(0, linewidth=1)
    ax.axvline(0, linewidth=1)
    ax.set_xlabel("x")
    ax.set_ylabel("y")

def arrow(ax, v, label=None):
    ax.arrow(0, 0, v[0], v[1], head_width=0.12, length_includes_head=True)
    if label:
        ax.text(v[0]*1.05, v[1]*1.05, label)

def draw_frame_2d(ax, origin, theta, s=0.8, label=None):
    o = np.array(origin, dtype=float)
    c, sn = np.cos(theta), np.sin(theta)
    ex = np.array([c, sn])
    ey = np.array([-sn, c])
    ax.arrow(o[0], o[1], s*ex[0], s*ex[1], head_width=0.10, length_includes_head=True)
    ax.arrow(o[0], o[1], s*ey[0], s*ey[1], head_width=0.10, length_includes_head=True)
    if label:
        ax.text(o[0] + 0.05, o[1] + 0.05, label)


# 1) Vectors in 2D

A vector is a **direction and magnitude**. In robotics, vectors represent:
- positions (points) like $p = \begin{bmatrix} x \\ y \end{bmatrix}$
- velocities, forces, axes of motion, etc.

Use sliders to explore vector addition and scaling.


In [ ]:
def show_vectors(scale_a=1.0, ax=0.8, ay=1.2, bx=1.0, by=0.5):
    a = np.array([ax, ay]) * scale_a
    b = np.array([bx, by])
    c = a + b

    fig, axp = plt.subplots(figsize=(6,6))
    setup_2d(axp, lim=3)
    arrow(axp, a, "a")
    arrow(axp, b, "b")
    arrow(axp, c, "a+b")
    axp.set_title("Vector addition and scaling")
    axp.text(-2.8, 2.5, f"a = {a.round(3)}\n b = {b.round(3)}\n a+b = {c.round(3)}",
             bbox=dict(boxstyle="round", alpha=0.15))
    plt.show()

ui = widgets.VBox([
    widgets.FloatSlider(value=1.0, min=-2, max=2, step=0.05, description="scale(a)"),
    widgets.FloatSlider(value=0.8, min=-2, max=2, step=0.05, description="a_x"),
    widgets.FloatSlider(value=1.2, min=-2, max=2, step=0.05, description="a_y"),
    widgets.FloatSlider(value=1.0, min=-2, max=2, step=0.05, description="b_x"),
    widgets.FloatSlider(value=0.5, min=-2, max=2, step=0.05, description="b_y"),
])
out = widgets.interactive_output(show_vectors, {
    "scale_a": ui.children[0],
    "ax": ui.children[1],
    "ay": ui.children[2],
    "bx": ui.children[3],
    "by": ui.children[4],
})
display(ui, out)


# 2) Dot product and angle

The dot product is:

$$
a \cdot b = \|a\|\,\|b\|\cos\theta
$$

Key uses in kinematics:
- computing angles between vectors/axes
- projection of one vector onto another
- checking orthogonality (dot product = 0)

Interactively explore how $a \cdot b$ changes with the angle.


In [ ]:
def dot_demo(a_len=1.5, b_len=1.5, theta_deg=45):
    theta = np.deg2rad(theta_deg)
    a = np.array([a_len, 0.0])
    b = b_len * np.array([np.cos(theta), np.sin(theta)])
    dot = float(a @ b)
    cos_th = dot / (np.linalg.norm(a)*np.linalg.norm(b) + 1e-12)

    proj_len = dot / (np.linalg.norm(a) + 1e-12)
    proj = np.array([proj_len, 0.0])

    fig, axp = plt.subplots(figsize=(6,6))
    setup_2d(axp, lim=3)
    arrow(axp, a, "a")
    arrow(axp, b, "b")
    axp.plot([b[0], proj[0]], [b[1], proj[1]], linestyle="--", linewidth=1)
    arrow(axp, proj, "proj(b onto a)")
    axp.set_title("Dot product and projection")
    axp.text(-2.8, 2.5,
             f"a·b = {dot:.3f}\ncosθ = {cos_th:.3f}\nθ = {theta_deg:.1f}°",
             bbox=dict(boxstyle="round", alpha=0.15))
    plt.show()

ui = widgets.VBox([
    widgets.FloatSlider(value=1.5, min=0.2, max=2.5, step=0.05, description="|a|"),
    widgets.FloatSlider(value=1.5, min=0.2, max=2.5, step=0.05, description="|b|"),
    widgets.FloatSlider(value=45, min=-180, max=180, step=1, description="θ (deg)"),
])
out = widgets.interactive_output(dot_demo, {
    "a_len": ui.children[0],
    "b_len": ui.children[1],
    "theta_deg": ui.children[2],
})
display(ui, out)


# 3) Matrices and matrix multiplication

A 2×2 matrix $A$ maps a vector $v$ to a new vector:

$$
v' = A v
$$

Matrix multiplication is **composition** of linear maps:
$$
A(Bv) = (AB)v
$$

Below, manipulate a 2×2 matrix and see how it transforms a grid and basis vectors.


In [ ]:
def lin_transform_demo(a11=1, a12=0, a21=0, a22=1):
    A = np.array([[a11, a12],
                  [a21, a22]], dtype=float)

    xs = np.linspace(-2, 2, 9)
    ys = np.linspace(-2, 2, 9)
    grid = np.array([[x, y] for x in xs for y in ys])
    grid_t = (A @ grid.T).T

    e1 = np.array([1,0])
    e2 = np.array([0,1])
    e1t = A @ e1
    e2t = A @ e2

    fig, axp = plt.subplots(figsize=(6,6))
    setup_2d(axp, lim=3)
    axp.scatter(grid[:,0], grid[:,1], s=10, alpha=0.35)
    axp.scatter(grid_t[:,0], grid_t[:,1], s=10, alpha=0.35)

    arrow(axp, e1, "e1")
    arrow(axp, e2, "e2")
    arrow(axp, e1t, "A e1")
    arrow(axp, e2t, "A e2")

    axp.set_title("Linear transform v' = A v (grid before/after)")
    axp.text(-2.8, 2.5, f"A =\n{np.round(A,3)}\n det(A) = {np.linalg.det(A):.3f}",
             bbox=dict(boxstyle="round", alpha=0.15))
    plt.show()

sliders = [
    widgets.FloatSlider(value=1, min=-2, max=2, step=0.05, description="a11"),
    widgets.FloatSlider(value=0, min=-2, max=2, step=0.05, description="a12"),
    widgets.FloatSlider(value=0, min=-2, max=2, step=0.05, description="a21"),
    widgets.FloatSlider(value=1, min=-2, max=2, step=0.05, description="a22"),
]
ui = widgets.VBox(sliders)
out = widgets.interactive_output(lin_transform_demo, {
    "a11": sliders[0], "a12": sliders[1], "a21": sliders[2], "a22": sliders[3]
})
display(ui, out)


# 4) Rotation matrices (2D)

A 2D rotation by angle $\theta$ is:

$$
R(\theta) =
\begin{bmatrix}
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta
\end{bmatrix}
$$

Rotations preserve lengths and angles: $\|Rv\| = \|v\|$.


In [ ]:
def rot2(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],
                     [s,  c]])

def rot_demo(theta_deg=30, vx=1.2, vy=0.6):
    theta = np.deg2rad(theta_deg)
    v = np.array([vx, vy])
    v2 = rot2(theta) @ v

    fig, axp = plt.subplots(figsize=(6,6))
    setup_2d(axp, lim=3)
    arrow(axp, v, "v")
    arrow(axp, v2, "R v")
    axp.set_title("2D rotation")
    axp.text(-2.8, 2.5, f"θ={theta_deg:.1f}°\n|v|={np.linalg.norm(v):.3f}\n|Rv|={np.linalg.norm(v2):.3f}",
             bbox=dict(boxstyle="round", alpha=0.15))
    plt.show()

ui = widgets.VBox([
    widgets.FloatSlider(value=30, min=-180, max=180, step=1, description="θ (deg)"),
    widgets.FloatSlider(value=1.2, min=-2, max=2, step=0.05, description="v_x"),
    widgets.FloatSlider(value=0.6, min=-2, max=2, step=0.05, description="v_y"),
])
out = widgets.interactive_output(rot_demo, {
    "theta_deg": ui.children[0], "vx": ui.children[1], "vy": ui.children[2]
})
display(ui, out)


# 5) Homogeneous transforms in 2D (rigid-body transforms)

A rigid transform combines rotation + translation. In 2D we can write:

$$
T =
\begin{bmatrix}
R & t \\
0\ 0 & 1
\end{bmatrix}
=
\begin{bmatrix}
\cos\theta & -\sin\theta & t_x \\
\sin\theta & \cos\theta & t_y \\
0 & 0 & 1
\end{bmatrix}
$$

To transform a point $p = [x\ y]^T$, use homogeneous coordinates:

$$
\tilde{p} = \begin{bmatrix} x \\ y \\ 1 \end{bmatrix},\quad
\tilde{p}' = T\,\tilde{p}.
$$


In [ ]:
def T2(theta, tx, ty):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, tx],
                     [s,  c, ty],
                     [0,  0,  1]], dtype=float)

def apply_T2(Tm, p):
    ph = np.array([p[0], p[1], 1.0])
    q = Tm @ ph
    return q[:2]

def demo_T2(theta_deg=30, tx=0.5, ty=0.2, px=1.0, py=0.5, show_frames=True):
    theta = np.deg2rad(theta_deg)
    T_AB = T2(theta, tx, ty)

    p_B = np.array([px, py])
    p_A = apply_T2(T_AB, p_B)

    fig, axp = plt.subplots(figsize=(6,6))
    setup_2d(axp, lim=3)

    if show_frames:
        draw_frame_2d(axp, origin=(0,0), theta=0.0, label="{A}")
        draw_frame_2d(axp, origin=(tx,ty), theta=theta, label="{B}")

    axp.scatter([p_A[0]], [p_A[1]], s=80)
    axp.text(p_A[0]+0.05, p_A[1]+0.05, "p (in A)")

    axp.set_title("2D rigid transform:  p_A = ( ^A T_B ) p_B")
    axp.text(-2.8, 2.5,
             f"θ={theta_deg:.1f}°, t=({tx:.2f},{ty:.2f})\n"
             f"p_B=({p_B[0]:.2f},{p_B[1]:.2f})\n"
             f"p_A=({p_A[0]:.2f},{p_A[1]:.2f})",
             bbox=dict(boxstyle="round", alpha=0.15))
    plt.show()

ui = widgets.VBox([
    widgets.FloatSlider(value=30, min=-180, max=180, step=1, description="θ (deg)"),
    widgets.FloatSlider(value=0.5, min=-2, max=2, step=0.05, description="t_x"),
    widgets.FloatSlider(value=0.2, min=-2, max=2, step=0.05, description="t_y"),
    widgets.FloatSlider(value=1.0, min=-2, max=2, step=0.05, description="p_x in B"),
    widgets.FloatSlider(value=0.5, min=-2, max=2, step=0.05, description="p_y in B"),
    widgets.Checkbox(value=True, description="Show frames"),
])
out = widgets.interactive_output(demo_T2, {
    "theta_deg": ui.children[0], "tx": ui.children[1], "ty": ui.children[2],
    "px": ui.children[3], "py": ui.children[4], "show_frames": ui.children[5]
})
display(ui, out)


## Changing coordinates vs moving points

If $T = {}^{A}T_{B}$, then multiplying $p_B$ gives the **same physical point** expressed in frame A:
$$
p_A = {}^{A}T_{B}\,p_B.
$$

The inverse transform converts coordinates the other way:
$$
p_B = ({}^{A}T_{B})^{-1} p_A = {}^{B}T_{A}\,p_A.
$$


# 6) Homogeneous transforms in 3D (interactive)

In 3D, rigid transforms use 4×4 matrices:

$$
{}^{A}T_{B} =
\begin{bmatrix}
{}^{A}R_{B} & {}^{A}p_{B} \\
0\ 0\ 0 & 1
\end{bmatrix}
$$

We’ll visualize a frame {B} rotated and translated relative to {A}, and a point expressed in {B}.  
You will see its coordinates in {A}.

We use a yaw–pitch–roll rotation:
$$
R = R_z(\text{yaw})\,R_y(\text{pitch})\,R_x(\text{roll}).
$$


In [ ]:
def Rx(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[1,0,0],[0,c,-s],[0,s,c]])

def Ry(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c,0,s],[0,1,0],[-s,0,c]])

def Rz(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]])

def T3(R, t):
    Tm = np.eye(4)
    Tm[:3,:3] = R
    Tm[:3,3] = t
    return Tm

def apply_T3(Tm, p):
    ph = np.array([p[0], p[1], p[2], 1.0])
    q = Tm @ ph
    return q[:3]

def frame_traces_3d(origin, R, axis_len=0.5, name_prefix=""):
    o = origin
    ex, ey, ez = R[:,0], R[:,1], R[:,2]
    xt, yt, zt = o + axis_len*ex, o + axis_len*ey, o + axis_len*ez
    traces = []
    traces.append(go.Scatter3d(x=[o[0], xt[0]], y=[o[1], xt[1]], z=[o[2], xt[2]], mode="lines"))
    traces.append(go.Scatter3d(x=[o[0], yt[0]], y=[o[1], yt[1]], z=[o[2], yt[2]], mode="lines"))
    traces.append(go.Scatter3d(x=[o[0], zt[0]], y=[o[1], zt[1]], z=[o[2], zt[2]], mode="lines"))
    traces.append(go.Scatter3d(x=[xt[0]], y=[xt[1]], z=[xt[2]], mode="text", text=["x"], showlegend=False))
    traces.append(go.Scatter3d(x=[yt[0]], y=[yt[1]], z=[yt[2]], mode="text", text=["y"], showlegend=False))
    traces.append(go.Scatter3d(x=[zt[0]], y=[zt[1]], z=[zt[2]], mode="text", text=["z"], showlegend=False))
    return traces

fig = go.FigureWidget()
fig.update_layout(
    title="3D rigid transform: p_A = ( ^A T_B ) p_B",
    scene=dict(
        xaxis=dict(range=[-3,3], title="x"),
        yaxis=dict(range=[-3,3], title="y"),
        zaxis=dict(range=[-3,3], title="z"),
        aspectmode="cube"
    ),
    margin=dict(l=0,r=0,t=40,b=0),
    showlegend=False
)

# Frame A (fixed)
for tr in frame_traces_3d(np.zeros(3), np.eye(3), axis_len=0.8):
    fig.add_trace(tr)
# Frame B (will update)
for tr in frame_traces_3d(np.array([1.0,0,0]), np.eye(3), axis_len=0.8):
    fig.add_trace(tr)

# Point p in A
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0], mode="markers+text", text=["p"], textposition="top center"))

roll = widgets.FloatSlider(value=0, min=-180, max=180, step=1, description="roll (deg)")
pitch = widgets.FloatSlider(value=0, min=-180, max=180, step=1, description="pitch (deg)")
yaw = widgets.FloatSlider(value=0, min=-180, max=180, step=1, description="yaw (deg)")
tx = widgets.FloatSlider(value=1.0, min=-2.0, max=2.0, step=0.05, description="t_x")
ty = widgets.FloatSlider(value=0.0, min=-2.0, max=2.0, step=0.05, description="t_y")
tz = widgets.FloatSlider(value=0.0, min=-2.0, max=2.0, step=0.05, description="t_z")
px = widgets.FloatSlider(value=0.8, min=-2.0, max=2.0, step=0.05, description="p_x in B")
py = widgets.FloatSlider(value=0.2, min=-2.0, max=2.0, step=0.05, description="p_y in B")
pz = widgets.FloatSlider(value=0.3, min=-2.0, max=2.0, step=0.05, description="p_z in B")

out = widgets.Output()

def update_3d(roll_deg, pitch_deg, yaw_deg, txv, tyv, tzv, pxv, pyv, pzv):
    r = np.deg2rad(roll_deg)
    p = np.deg2rad(pitch_deg)
    y = np.deg2rad(yaw_deg)
    R = Rz(y) @ Ry(p) @ Rx(r)
    t = np.array([txv, tyv, tzv])
    T_AB = T3(R, t)

    p_B = np.array([pxv, pyv, pzv])
    p_A = apply_T3(T_AB, p_B)

    with fig.batch_update():
        B_start = 6  # after A frame traces
        B_trs = frame_traces_3d(t, R, axis_len=0.8)
        for i in range(6):
            fig.data[B_start + i].x = B_trs[i].x
            fig.data[B_start + i].y = B_trs[i].y
            fig.data[B_start + i].z = B_trs[i].z
            fig.data[B_start + i].mode = B_trs[i].mode
            fig.data[B_start + i].text = getattr(B_trs[i], "text", None)

        fig.data[-1].x = [p_A[0]]
        fig.data[-1].y = [p_A[1]]
        fig.data[-1].z = [p_A[2]]

    with out:
        out.clear_output()
        print("p_B =", np.round(p_B, 4))
        print("p_A =", np.round(p_A, 4))
        print("^A T_B (rounded) =")
        print(np.round(T_AB, 4))

ui = widgets.VBox([widgets.HBox([roll, pitch, yaw]), widgets.HBox([tx, ty, tz]), widgets.HBox([px, py, pz])])
widgets.interactive_output(update_3d, {
    "roll_deg": roll, "pitch_deg": pitch, "yaw_deg": yaw,
    "txv": tx, "tyv": ty, "tzv": tz,
    "pxv": px, "pyv": py, "pzv": pz
})
display(ui, fig, out)
update_3d(roll.value, pitch.value, yaw.value, tx.value, ty.value, tz.value, px.value, py.value, pz.value)


# 7) Practice questions (quick)

1. In 2D, set $\theta=90^\circ$, $t=(1,0)$, and $p_B=(1,0)$. What is $p_A$? Verify with the widget.
2. In 3D, keep translation $t=0$ and rotate yaw from $0$ to $90^\circ$. Which axis does the point rotate around in world coordinates?
3. Compute $p_B = ({}^{A}T_{B})^{-1} p_A$ for a point you choose. (Hint: for rigid transforms, $R^{-1} = R^T$.)

If you want, we can extend this refresher into cross products, orthonormal frames, and Jacobians.
